                                        Додаткове завдання № 7 з зірочкою

Для більшого заглиблення в роботу алгоритму, пропонуємо реалізувати алгоритм колабораційної фільтрації з нуля. Для цього ми можемо скористатись нашою домашньою роботою з 3-ого модуля. Якщо ми модифікуємо функцію втрат та розрахунок градієнтів, то зможемо побудувати алгоритм матричної факторизації.

Додамо потрібні імпорти:

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.optimize import minimize

Добре, для цього для початку давайте завантажимо файли movie_ids.txt та movies.mat

In [3]:
def loadMovieList():
    with open('movie_ids.txt', encoding='ISO-8859-1') as fid:
        movies = fid.readlines()

    movieNames = []
    for movie in movies:
        parts = movie.split()
        movieNames.append(' '.join(parts[1:]).strip())
    return movieNames

In [5]:
names = loadMovieList()
names[:10]

['Toy Story (1995)',
 'GoldenEye (1995)',
 'Four Rooms (1995)',
 'Get Shorty (1995)',
 'Copycat (1995)',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 'Twelve Monkeys (1995)',
 'Babe (1995)',
 'Dead Man Walking (1995)',
 'Richard III (1995)']

In [6]:
data = loadmat('movies.mat')
Y, R = data['Y'], data['R']

print('Average rating for movie 1601 (',names[1600] ,'): %f / 5' %
      np.mean(Y[1600, R[0, :]]))

Average rating for movie 1601 ( Office Killer (1997) ): 0.000000 / 5


Для початку нам потрібно ініціалізувати наші параметри. Перш ніж ми почнемо з функцією втрат, нам потрібно ініціалізувати наші параметри 
X та Θ. Давайте припустимо, що ми хочемо мати n=10(наш гіперпараметр) особливостей для кожного фільму та користувача:

In [7]:
num_movies, num_users = Y.shape
num_features = 10

X = np.random.rand(num_movies, num_features)
Theta = np.random.rand(num_users, num_features)

Далі будемо реалізовувати функції втрат без регуляризації і далі вже подивимось, чи потрібна нам регуляризація.

In [8]:
def computeCost(X, Theta, Y, R):
    """
    X - матриця особливостей фільмів, розміром (num_movies x num_features)
    Theta - матриця параметрів користувачів, розміром (num_users x num_features)
    Y - матриця рейтингів, розміром (num_movies x num_users)
    R - матриця, де r(i, j) = 1, якщо користувач j оцінив фільм i
    """
    # Розрахунок помилки прогнозу(формула наших втрат)
    error = (X @ Theta.T - Y) * R
    # Втрата без регуляризації
    J = 1/2 * np.sum(error**2)
    return J

Тепер потрібно обчислити градієнт для параметрів X і Θ:

In [9]:
def computeGradient(X, Theta, Y, R):
    error = (X @ Theta.T - Y) * R
    X_grad = error @ Theta
    Theta_grad = error.T @ X
    return X_grad, Theta_grad

Тепер, коли у нас є функції для обчислення втрат та градієнтів, наступний крок – це оптимізація параметрів X і Θ за допомогою scipy.optimize.minimize.

Потрібно модифікувати наші функції, щоб вони приймали параметри у вигляді одновимірного масиву, а також додати параметр регуляризації

In [10]:
def computeCost(params, Y, R, num_users, num_movies, num_features, lambda_):
    X = params[:num_movies*num_features].reshape(num_movies, num_features)
    Theta = params[num_movies*num_features:].reshape(num_users, num_features)
    error = (X @ Theta.T - Y) * R
    cost = 1/2 * np.sum(error**2) + (lambda_/2) * (np.sum(Theta**2) + np.sum(X**2))
    return cost

def computeGradient(params, Y, R, num_users, num_movies, num_features, lambda_):
    X = params[:num_movies*num_features].reshape(num_movies, num_features)
    Theta = params[num_movies*num_features:].reshape(num_users, num_features)
    error = (X @ Theta.T - Y) * R
    X_grad = error @ Theta + lambda_ * X
    Theta_grad = error.T @ X + lambda_ * Theta
    grad = np.concatenate([X_grad.ravel(), Theta_grad.ravel()])
    return grad


Далі проводимо оптимізацію за допомогою пакету scipy.optimize.minimize:

In [11]:
# Параметри
lambda_ = 10
params = np.concatenate([X.ravel(), Theta.ravel()])

# Оптимізація
res = minimize(fun=computeCost, x0=params, args=(Y, R, num_users, num_movies, num_features, lambda_),
               method='TNC', jac=computeGradient, options={'maxfun': 100})

# Отримані оптимізовані параметри
params_opt = res.x

# Розпаковка оптимізованих параметрів
X_opt = params_opt[:num_movies*num_features].reshape(num_movies, num_features)
Theta_opt = params_opt[num_movies*num_features:].reshape(num_users, num_features)


Після того, як ми отримали оптимізовані матриці особливостей фільмів X та параметри користувачів Θ, ми можемо використовувати ці дані для генерації рекомендацій

In [12]:
P = np.dot(X_opt, Theta_opt.T)

Вибираємо індекс користувача, для якого хочимо згенерувати рекомендації. Наприклад, для користувач під індексом 5:

In [13]:
user_index = 5

Далі для отримання рекомендацій нам потрібно відсортувати фільми за прогнозованим рейтингом для обраного користувача:

In [14]:
user_ratings = Y[:, user_index]
user_predictions = P[:, user_index]

# Фільтруємо фільми, які користувач уже оцінив
unrated_movies = np.where(user_ratings == 0)[0]

# Прогнози для непереглянутих фільмів
unrated_predictions = user_predictions[unrated_movies]

# Отримання індексів фільмів з найвищими прогнозами
top_n = 10
recommended_movie_idxs = np.argsort(-unrated_predictions)[:top_n]

# Виводимо топ-10 рекомендованих фільмів користувачу
recommended_movies = [names[i] for i in recommended_movie_idxs]
print(f"Рекомендовані фільми для користувача: {user_index}")
for movie in recommended_movies:
    print(movie)


Рекомендовані фільми для користувача: 5
Birds, The (1963)
Man Without a Face, The (1993)
In & Out (1997)
Burnt Offerings (1976)
Duck Soup (1933)
His Girl Friday (1940)
Desperado (1995)
Jaws 2 (1978)
Star Trek V: The Final Frontier (1989)
American Werewolf in London, An (1981)
